# Simluating a population of stars with dust extinction

The MegaBEAST (MB) can be used to simulate observations of a population of stars extinguished by dust.

The MB uses a hierarchical Bayesian framework.

The 1st most level is provided by the BEAST that provides models of individual stars.  The BEAST model is based on 7 parameters.  The stellar parameters are age (logA), initial mass (M_ini), metallicity (Z), and distance (distance).  The dust extinction parameters are dust column (Av), average dust size (Rv), and MW/SMC composition (f_A).  For the details see Gordon et al. (2016) and the BEAST docs.

The MB uses the BEAST physics and observation models to perform the simulations.  The MB physics model describes the stellar population and dust extinction distributions.  The MB stellar age distribution is given by the `logA` model (often star formation rate in specific age bins), the `M_ini` model (e.g, initital mass function), the `Z` model (often relative values in specific age bins), and the `distance` model (e.g., flat or absolute exponential center on a specific distance).  The MB dust distribution is given by the `Av`, `Rv`, and `f_A` models that are each often two lognormals - one for internal dust and one for foreground dust.

The MB physics model is the equivalent of the BEAST prior model.  In fact, the MB uses the BEAST prior model code to calculate the MB physics model.  All the possible MB distributions are documented at https://beast.readthedocs.io/en/latest/beast_priors.html

## Specifying the MegaBEAST physics model

The MB physics model is specified using two python dictonaries.

### MB stellar physics model

In [1]:
# stellar population model
stellar_model = {
    "name": "bins_histo",
    "x": [6.0, 7.0, 8.0, 9.0, 10.0],  # units are log(years)
    "logA": {  # star formation history SFH
        "name": "bins_histo",
        "varnames": ["values"],
        "varinit": [[1e-8, 1e-8, 1e-8, 1e-8, 1e-8]],  # units are M_sun/year
        "prior": {
            "name": "fixed",
            "minmax": [[0.0, 0.0, 0.0, 0.0, 0.0], [1e-3, 1e-3, 1e-3, 1e-3, 1e-3]],
        },
    },
    "M_ini": {  # initial mass function
        "name": "salpeter",
        "varnames": ["slope"],
        "varinit": [2.35],
        "prior": {
          "name": "fixed",
          "minmax": [[2.0, 3.0]],
        }
    },
    "Z": {
        "name": "bins_histo",
        "init": [-0.3, -0.3, -0.3, -0.3, -0.3],
        "prior": {
            "name": "fixed",
            "min": [-2.1, -2.1, -2.1, -2.1, -2.1],
            "max": [0.0, 0.0, 0.0, 0.0, 0.0],
        },
    },
    "distance": {
        "name": "Gaussian",
        "m_unit": "kpc",
        "m_init": 60.0,  # units are
        "s_init": 5.0,
        "d_prior": {"name": "flat", "min": 50.0, "max": 70.0},
        "s_prior": {"name": "flat", "min": 1.0, "max": 10.0},
    },
}


### MB dust extinction model

This example is for a foreground screen of dust.  (should update to a more realistic foreground plus internal dust)

In [2]:
# foreground dust cloud
dust_model = {
    "Av": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [1.0, 0.1],
        "prior": {
            "name": "flat",
            "minmax": [[0.005, 5.0], [0.05, 1.0]],
        },
    },
    "Rv": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [3.1, 0.25],
        "prior": {
            # "name": "fixed",
            "name": "flat",
            "minmax": [[2.0, 6.0], [0.05, 1.0]],
        },
    },
    "f_A": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [1.0, 0.25],
        "prior": {
            "name": "fixed",
            "minmax": [[0.0, 1.0], [0.05, 0.5]],
        },
    }
}


### setup MegaBEAST params

The two dictonaries can be provide in a text file or in code directly.  This notebook uses the code option.

In [3]:
from megabeast.mbsettings import mbsettings

mbparams = mbsettings()
mbparams.dict_to_attr(stellar_model | dust_model)